In [0]:
#-*-coding:utf-8 -*-

###################################### IMPORTATIONS, TOUT CA TOUT CA ######################################


!pip install -U -q PyDrive

import numpy as np
import csv
import os

from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
from google.colab import files

In [0]:
# code trouvé sur Internet
# création de fichier csv et sauvegarde

def bootstrap(tailleBase = 5000) :
  
  ###################################### TELECHARGEMENT DE LA BASE GLOBALE ######################################
  
  
  # Authentification et initialisation du client PyDrive
  auth.authenticate_user()
  gauth = GoogleAuth()
  gauth.credentials = GoogleCredentials.get_application_default()
  drive = GoogleDrive(gauth)

  # Choix d'un répertoire distant pour charger les données
  # Pour rester à la racine, mettre seulement "~" dans le expanduser
  # Pour se placer dans un dossier toto situé à la racine, mettre '~/toto' dans le expanduser
  # Pour se placer dans un dossier toto situé dans un dossier titi situé à la racine, mettre '~/titi/toto' dans le expanduser
  # and so on...
  local_download_path = os.path.expanduser('~')
  try:
    os.makedirs(local_download_path)
  except: pass

  # Ciblage du dossier drive contenant les données
  folderID = "1ggpBuvJLQir2dJuNYuoYgcCjPADgg82O" # A MODIFIER
  file_list = drive.ListFile({'q': "'{}' in parents".format(folderID)}).GetList()

  for f in file_list:
    print('title: %s, id: %s' % (f['title'], f['id']))
    fname = os.path.join(local_download_path, f['title'])
    #fname = f['title']
    print('downloading to {}'.format(fname))
    f_ = drive.CreateFile({'id': f['id']})
    f_.GetContentFile(fname)
    
    
  ###################################### CREATION DE LA BASE BOOTSTRAPEE ######################################    
  
  
  with open('BDDBootstrap.csv', 'wb') as bddbootstrap : # Création du fichier destiné à contenir notre BDD bootstrappée
    csvwriter = csv.writer(bddbootstrap, dialect = 'excel')
    csvwriter.writerow(["pixel"+str(ind) for ind in range(1024)]+["shape","color","x_centre","y_centre","rotation","size","elongation"])
    
    with open('BDDBabyAIAll_train.csv', 'rb') as bddall : # BDD contenant tous les exemples de train
      lines = bddall.readlines()
      
      listeIndices = list() # Création de la liste contenant tous les indices des lignes que nous sélectionnerons dans la BDD générale pour les mettre dans la BDD bootstrappée
      
      for i in range(tailleBase) :  # Peut-être tailleBase+1 ? A tester
        listeIndices.append(np.random.randint(160000)) # 160000 = taille de la base globale que nous avons créée, contenant tous les exemples
        line = bddall[listeIndices[i]]
        line_list = line.strip('\#size:').split()
        csvwriter.writerow(line_list)
   
  ###################################### TELECHARGEMENT DE LA BASE BOOTSTRAPEE ######################################      
  
    files.download('BDDBootstrap.csv') # Si on pouvait trouver comment la mettre directement sur le Drive, ce serait bien
    

En gros, quand j'utilise pd.read_csv, il va avaler la première ligne du fichier csv, en créant une matrice de taille (nombre de linge **qui reste**, nombre de colone de la **ligne que la méthode a gobée**). Donc en rajoutant une ligne de zero je peux créer une matrice de taille (5001, 1031) où la première ligne ne contient que 2 élément, donc 5000 et 1031. Si je ne rajoute pas cette ligne de zero, j'obtiendrai une matrice de taille (5000,2)

In [0]:
X_train.shape # Petit test des familles

(5000, 32, 32, 1)